# Sporulation

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/me_modeling/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/__init__.py
/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/ecolime-master/ecolime/__init__.py


In [22]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f79da2e9898>

In [23]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

### Is CS essential for growth? (citZ)

In [15]:
me.reactions.get_by_id('CS_FWD_BSU29140-MONOMER').upper_bound=0

In [16]:
me.reactions.get_by_id('CS_FWD_BSU09440-MONOMER').upper_bound=0

In [17]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-6)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 206.843772 seconds
Finished substituting S,lb,ub in 9.316815 seconds
Finished makeME_LP in 1.050280 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 8.988180 seconds
Finished makeME_LP in 1.024608 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 8.833023 seconds
Finished makeME_LP in 1.023247 seconds
Getting MINOS parameters from ME_NLP...
3 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 8.823088 seconds
Finished makeME_LP in 1.033025 seconds
Getting MINOS parameters from ME_NLP...
4 0.0625 0.0625 0.09375 0.09375 1
Finished substituting S,lb,ub in 8.736071 seconds
Finished makeME_LP in 1.016968 seconds
Getting MINOS parameters from ME_NLP...
5 0.078125 0.078125 0.09375 0.078125 optimal
Finished substituting S,lb,ub in 8.684958 seconds
Finished makeME_LP in 1.020726 seconds
Getti

In [ ]:
flux_dict = me.get_metabolic_flux()
pd.DataFrame.from_dict({'flux':flux_dict})#.to_csv('../fluxdist_CS-.csv')

In [18]:
rxn_dict,rxn_id_dict = homogenize_reactions(bsub,eco)

100%|██████████| 1250/1250 [00:00<00:00, 4649.18it/s]


In [9]:
import pandas as pd
FS_df = pd.read_csv('essentiality_raw_results_FS.csv',index_col=0).T

In [10]:
def get_binary_essentiality(df,threshold = 1e-3,obj = 'biomass_dilution'):
    d = {}
    for g,row in df.T.iterrows():
        d[g.split('_s')[0]] = 1 if row[obj] < threshold else 0
    return d
def get_comparison(l,annotation=[],names=[]):
    d = {}
    for i in l[0]:
        if any(i not in di for di in l):
            continue
        v = [di[i] for di in l]
        if not names:
            names = range(len(l))
        d[i] = {ni:vi for ni,vi in zip(names,v)}
        if isinstance(annotation,pd.DataFrame):
            p = '{}-MONOMER'.format(i)
            if p in annotation.index:
                d[i]['annotation'] = annotation.loc[p]['name']
    return pd.DataFrame.from_dict(d).T

In [11]:
FS_es = get_binary_essentiality(FS_df.T,obj = 'biomass_dilution_s')

In [12]:
FS_es['BSU15680']

0